In [ ]:
# !pip install PyPDF2
# !pip install datasets
# !pip install evaluate
# !pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinu

In [ ]:
#New Prediction

import os
import torch
from PyPDF2 import PdfReader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()



def predict_and_eavluate(test_dir = None, test_file = None, model = None, tokenizer = None,label_encoder_path = None):

    # Define a function to extract text from a PDF file
    def extract_text_from_pdf(pdf_path):
        text = ""
        with open(pdf_path, "rb") as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text()
        return text.strip()


    with open(label_encoder_path, "rb") as f:
         label_encoder = pickle.load(f)

    if test_file != None:
       new_text = extract_text_from_pdf(test_file)
       inputs = tokenizer(new_text, padding="max_length", truncation=True, return_tensors="pt")
       # Perform inference
       with torch.no_grad():
            outputs = model(**inputs)

       # Get the predicted label
       predicted_label = torch.argmax(outputs.logits).item()

       # Decode the predicted label
       label_text = label_encoder.inverse_transform([predicted_label])[0]
       print("Predicted Label:", label_text)


    if test_dir != None:
        # Define the root directory containing subdirectories with PDF files
        root_directory = test_dir

        # Initialize lists to store true labels and predicted labels
        true_labels = []
        predicted_labels = []

        # Iterate over all subdirectories in the root directory
        for subdir in os.listdir(root_directory):
            subdir_path = os.path.join(root_directory, subdir)
            if os.path.isdir(subdir_path):
                # Iterate over all PDF files in the subdirectory
                for filename in os.listdir(subdir_path):
                    if filename.endswith(".pdf"):
                        # Construct the full path to the PDF file
                        pdf_path = os.path.join(subdir_path, filename)

                        # Extract text from the PDF file
                        new_text = extract_text_from_pdf(pdf_path)

                        # Tokenize and preprocess the new text data
                        inputs = tokenizer(new_text, padding="max_length", truncation=True, return_tensors="pt")

                        # Perform inference
                        with torch.no_grad():
                            outputs = model(**inputs)

                        # Get the predicted label
                        predicted_label = torch.argmax(outputs.logits).item()
                        label_text = label_encoder.inverse_transform([predicted_label])[0]

                        # Record the true label (subdirectory name) and predicted label
                        true_labels.append(subdir)
                        predicted_labels.append(label_text)

        # Print classification report based on true labels and predicted labels
        print(classification_report(true_labels, predicted_labels))


        class_labels = sorted(os.listdir(root_directory))
        conf_matrix = confusion_matrix(true_labels, predicted_labels, labels=class_labels)
        # Plot confusion matrix
        plt.figure(figsize=(10, 8))
        sns.set(font_scale=1.2)
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
        plt.xlabel("Predicted Label")
        plt.ylabel("True Label")
        plt.title("Confusion Matrix")
        plt.show()



In [ ]:
#Prediction
# Load the saved model and tokenizer
model_root_dir = "/content/drive/MyDrive/bert-80-20"
label_encoder_path = model_root_dir +"/label_encoder.pkl"
model_path = model_root_dir +"/saved_bert_model_80_20"

model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)


test_dir = "/content/drive/MyDrive/dataset/test_pdf"
test_file = "/content/drive/MyDrive/dataset/dataset_pdf/entertainment/text_10.pdf"

predict_and_eavluate(test_dir = test_dir, test_file = test_file, model= model, tokenizer= tokenizer,label_encoder_path= label_encoder_path)